In [2]:
import twitter_creds
import requests
import base64
from pprint import pprint

In [3]:
def get_bearer_header():
    uri_token_endpoint = 'https://api.twitter.com/oauth2/token'
    key_secret = f"{twitter_creds.consumer_key}:{twitter_creds.consumer_key_secret}".encode('ascii')
    b64_encoded_key = base64.b64encode(key_secret)
    b64_encoded_key = b64_encoded_key.decode('ascii')

    auth_headers = {
        'Authorization': 'Basic {}'.format(b64_encoded_key),
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
    }

    auth_data = {
        'grant_type': 'client_credentials'
    }

    auth_resp = requests.post(uri_token_endpoint, headers=auth_headers, data=auth_data)
    bearer_token = auth_resp.json()['access_token']

    bearer_header = {
       'Accept-Encoding': 'gzip',
       'Authorization': 'Bearer {}'.format(bearer_token),
       'oauth_consumer_key': twitter_creds.consumer_key 
    }
    return bearer_header

In [4]:
def getConversationId(id):
    uri = 'https://api.twitter.com/2/tweets?'

    params = {
        'ids':id,
        'tweet.fields':'conversation_id',
        'max_results': 100
    }

    bearer_header = get_bearer_header()
    resp = requests.get(uri, headers=bearer_header, params=params)
    return resp.json()['data'][0]['conversation_id']

In [39]:

#   --url 'https://api.twitter.com/2/tweets/search/recent?query=conversation_id:1279940000004973111&tweet.fields=in_reply_to_user_id,author_id,created_at,conversation_id' \
#   --header 'Authorization: Bearer $BEARER_TOKEN'  


def getConversation(conversation_id):
    data = {}
    shown = False
    uri = 'https://api.twitter.com/2/tweets/search/recent'

    params = {
        'query': f'conversation_id:{conversation_id}',
        'tweet.fields': 'in_reply_to_user_id,author_id,created_at,conversation_id', 
    }

    bearer_header = get_bearer_header()
    token = ''
    while token != None:
        if token != '':
            params['next_token'] = token
        resp = requests.get(uri, headers=bearer_header, params=params).json()
        if not shown:
            print(resp)
            shown = True
        try:
            token = resp['meta']['next_token']
        except:
            break
        if not data:
            # print("empty")
            data['data'] = resp['data']
        else:
            # print("updating")
            data['data'].extend(resp['data'])
        # print(resp['meta']['result_count'])
    return data

In [14]:
getConversationId(1478993643516018692)

'1478993643516018692'

In [40]:
conversations = getConversation(1478993643516018692)

{'data': [{'created_at': '2022-01-08T11:35:30.000Z', 'conversation_id': '1478993643516018692', 'text': '@XiaomiIndia When will you announce the winners??', 'id': '1479778785373290498', 'author_id': '927180770774368258', 'in_reply_to_user_id': '2646263522'}, {'created_at': '2022-01-08T10:29:17.000Z', 'conversation_id': '1478993643516018692', 'text': '@XiaomiIndia 💯🔥🔥\n\n#Xiaomi11iHyperCharge #HyperchargeRevolution #Xiaomi11i #Xiaomi11iSeries @XiaomiIndia @manukumarjain', 'id': '1479762124905742337', 'author_id': '1058371116136194048', 'in_reply_to_user_id': '2646263522'}, {'created_at': '2022-01-08T08:03:06.000Z', 'conversation_id': '1478993643516018692', 'text': "@XiaomiIndia Great spirit, that's the way to go! #HyperchargeRevolution\xa0 #Xiaomi11iHypercharge", 'id': '1479725335021121537', 'author_id': '1318969101599952896', 'in_reply_to_user_id': '2646263522'}, {'created_at': '2022-01-08T06:05:01.000Z', 'conversation_id': '1478993643516018692', 'text': '@XiaomiIndia Ready \n\n#Xiaomi1

In [41]:
conversations['data'][:10]

[{'created_at': '2022-01-08T11:35:30.000Z',
  'conversation_id': '1478993643516018692',
  'text': '@XiaomiIndia When will you announce the winners??',
  'id': '1479778785373290498',
  'author_id': '927180770774368258',
  'in_reply_to_user_id': '2646263522'},
 {'created_at': '2022-01-08T10:29:17.000Z',
  'conversation_id': '1478993643516018692',
  'text': '@XiaomiIndia 💯🔥🔥\n\n#Xiaomi11iHyperCharge #HyperchargeRevolution #Xiaomi11i #Xiaomi11iSeries @XiaomiIndia @manukumarjain',
  'id': '1479762124905742337',
  'author_id': '1058371116136194048',
  'in_reply_to_user_id': '2646263522'},
 {'created_at': '2022-01-08T08:03:06.000Z',
  'conversation_id': '1478993643516018692',
  'text': "@XiaomiIndia Great spirit, that's the way to go! #HyperchargeRevolution\xa0 #Xiaomi11iHypercharge",
  'id': '1479725335021121537',
  'author_id': '1318969101599952896',
  'in_reply_to_user_id': '2646263522'},
 {'created_at': '2022-01-08T06:05:01.000Z',
  'conversation_id': '1478993643516018692',
  'text': '@Xi